In [ ]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib
import matplotlib.pyplot as plt
from credentials import Credentials as cr
from prediction import ImmothepPrediction
import os
from split import Splitter

splitter = Splitter()

## Création et import du jeu de données principal

In [ ]:
valeurs2019 = pd.read_csv(cr.PATH+'2019.txt', sep='|', usecols=['Nature mutation', 'Code postal','Nombre pieces principales', 'Surface terrain', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot', 'Valeur fonciere', 'Type local', 'Nombre de lots'], encoding='utf-8')

valeurs2019 = valeurs2019.dropna(subset = ['Type local', 'Nombre de lots', 'Nombre pieces principales', "Nature mutation"])

valeurs2019['Surface terrain'][valeurs2019['Type local'].str.contains("Appartement", regex=True)] = valeurs2019['Surface terrain'][valeurs2019['Type local'].str.contains("Appartement", regex=True)].fillna(0) 

valeurs2019 = valeurs2019[['Code postal', 'Nombre pieces principales', 'Surface terrain', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot', 'Valeur fonciere']][valeurs2019['Type local'].str.contains("Appartement|Maison", regex=True)][valeurs2019['Nature mutation'].str.contains("Vente", regex=True)][valeurs2019['Nombre de lots']<6][valeurs2019['Nombre pieces principales']>0]

valeurs2019.head()

In [ ]:
# Vérification du ratio de valeurs nulles en vue de nettoyer les données.

nancount = valeurs2019["Valeur fonciere"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Valeur Foncière\'.")
valeurs2019 = valeurs2019.dropna(subset = ['Valeur fonciere'])
nancount = valeurs2019["Valeur fonciere"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Valeur Foncière\'.")

nancount = valeurs2019["Code postal"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Code postal\'.")
valeurs2019 = valeurs2019.dropna(subset = ['Code postal'])
nancount = valeurs2019["Code postal"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Code postal\'.")


In [ ]:
valeurs2019.info()

In [ ]:
valeurs2019[['Valeur fonciere', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot']] = valeurs2019[['Valeur fonciere', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot']].replace(',', '.', regex=True)

valeurs2019[['Valeur fonciere', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot']]  = valeurs2019[['Valeur fonciere', 'Surface Carrez du 1er lot', 'Surface Carrez du 2eme lot', 'Surface Carrez du 3eme lot', 'Surface Carrez du 4eme lot', 'Surface Carrez du 5eme lot']].astype('float')

In [ ]:
valeurs2019.info()

In [ ]:
corrMatrix = valeurs2019.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
valeurs2019 = valeurs2019[valeurs2019['Valeur fonciere']>200]

In [ ]:
nancount = valeurs2019["Surface terrain"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Surface Terrain\'.")
valeurs2019 = valeurs2019.dropna(subset = ['Surface terrain'])

nancount = valeurs2019["Surface terrain"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Surface Terrain\'.")

valeurs2019 = valeurs2019.fillna(0)

In [ ]:
corrMatrix = valeurs2019.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

Test de la prédiction de la valeur foncière sur l'élémant d'index 5 de valeurs2019

In [ ]:
predictor = ImmothepPrediction()

try:
    result = predictor.predict(valeurs2019, 5)
except : 
    print('Elément non trouvé.')

### Modification du modèle avec méthode d'entraînement et méthode de test

In [ ]:
modele, scaler, lab_enc = predictor.entrainementLR(valeurs2019)

result = predictor.predictionLR(valeurs2019[valeurs2019.index==693], modele, scaler, lab_enc)

In [ ]:
predictor.predictionLR(valeurs2019[valeurs2019.index==10], modele, scaler, lab_enc)

### Modification du dataset pour rendre la prédiction plus précise

In [ ]:
valeurs2019.plot.scatter("Surface terrain", "Valeur fonciere")

In [ ]:
valeurs2019 = valeurs2019[valeurs2019['Valeur fonciere']<60000000]
valeurs2019 = valeurs2019[valeurs2019['Surface terrain']<400000]

In [ ]:
valeurs2019.plot.scatter("Surface terrain", "Valeur fonciere")

### Nouveau test de prédiction

In [ ]:
modele, scaler, lab_enc = predictor.entrainementLR(valeurs2019)

In [ ]:
predictor.predictionLR(valeurs2019[valeurs2019.index==5], modele, scaler, lab_enc)

In [ ]:
predictor.predict(valeurs2019, 5)

In [ ]:
predictor.predict(valeurs2019, 693)

In [ ]:
predictor.predictionLR(valeurs2019[valeurs2019.index==693], modele, scaler, lab_enc)

### Export du premier dataset de travail

In [ ]:
valeurs2019.to_csv(cr.CURATED_LOCAL_PATH + 'filtered2019.csv', index=False)

## Création d'un nouveau dataset avec des attributs différents

In [ ]:
valeurs2019 = pd.read_csv(cr.PATH+'2019.txt', sep='|', usecols=['Nature mutation', 'Code postal','Nombre pieces principales', 'Surface terrain', 'Surface reelle bati', 'Valeur fonciere', 'Type local', 'Nombre de lots'], encoding='utf-8')

valeurs2019 = valeurs2019.dropna(subset = ['Type local', 'Nombre de lots', 'Nombre pieces principales', 'Nature mutation', 'Surface reelle bati'])

valeurs2019['Surface terrain'][valeurs2019['Type local'].str.contains("Appartement", regex=True)] = valeurs2019['Surface terrain'][valeurs2019['Type local'].str.contains("Appartement", regex=True)].fillna(0) 

valeurs2019 = valeurs2019[['Type local', 'Code postal', 'Nombre pieces principales', 'Surface terrain', 'Surface reelle bati', 'Valeur fonciere']][valeurs2019['Type local'].str.contains("Appartement|Maison", regex=True)][valeurs2019['Nature mutation'].str.contains("Vente", regex=True)][valeurs2019['Nombre de lots']][valeurs2019['Nombre pieces principales']>0]

valeurs2019.head()


In [ ]:
# Vérification du ratio de valeurs nulles en vue de nettoyer les données.

nancount = valeurs2019["Valeur fonciere"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Valeur Foncière\'.")
valeurs2019 = valeurs2019.dropna(subset = ['Valeur fonciere'])
nancount = valeurs2019["Valeur fonciere"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Valeur Foncière\'.")

nancount = valeurs2019["Code postal"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Code postal\'.")
valeurs2019 = valeurs2019.dropna(subset = ['Code postal'])
nancount = valeurs2019["Code postal"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Code postal\'.")


In [ ]:
valeurs2019.info()

In [ ]:
valeurs2019[['Valeur fonciere']] = valeurs2019[['Valeur fonciere']].replace(',', '.', regex=True)

valeurs2019[['Valeur fonciere']]  = valeurs2019[['Valeur fonciere']].astype('float')

In [ ]:
valeurs2019 = valeurs2019[valeurs2019['Valeur fonciere']>200]
valeurs2019 = valeurs2019[valeurs2019['Valeur fonciere']<60000000]
valeurs2019 = valeurs2019[valeurs2019['Surface terrain']<400000]

nancount = valeurs2019["Surface terrain"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Surface Terrain\'.")
valeurs2019 = valeurs2019.dropna(subset = ['Surface terrain'])
nancount = valeurs2019["Surface terrain"].isna()
print ("Il y a ", nancount.sum(), " valeurs nulles dans la colonne \'Surface Terrain\'.")

In [ ]:
maisons2019=valeurs2019[['Code postal', 'Nombre pieces principales', 'Surface terrain', 'Surface reelle bati', 'Valeur fonciere']][valeurs2019['Type local'].str.contains("Maison", regex=True)].reset_index(drop=True)

appart2019=valeurs2019[['Code postal', 'Nombre pieces principales', 'Surface terrain', 'Surface reelle bati', 'Valeur fonciere']][valeurs2019['Type local'].str.contains("Appartement", regex=True)][valeurs2019['Surface reelle bati']>9].reset_index(drop=True)

In [ ]:
maisons2019.plot.scatter("Surface terrain", "Valeur fonciere")

In [ ]:
appart2019.plot.scatter("Surface terrain", "Valeur fonciere")

In [ ]:
maisons2019.to_csv(cr.CURATED_LOCAL_PATH + 'filteredmaisons2019.csv', index=False)

appart2019.to_csv(cr.CURATED_LOCAL_PATH + 'filteredappart2019.csv', index=False)

In [ ]:
corrMatrix = maisons2019.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
corrMatrix = appart2019.corr()
sn.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
predictor = ImmothepPrediction()

modele_maison, scaler_maison, lab_enc_maison = predictor.entrainementLR(maisons2019)

In [ ]:
modele_appart, scaler_appart, lab_enc_appart = predictor.entrainementLR(appart2019)

In [ ]:
predictor.predictionLR(appart2019[appart2019.index==28], modele_maison, scaler_maison, lab_enc_maison)

In [ ]:
modele_lineaire_appart = predictor.entrainementLinear(appart2019)

In [ ]:
predictor.predictionLinear(appart2019[appart2019.index==28], modele_lineaire_appart)

In [ ]:
modele_lineaire_appart = predictor.trainLinearLeRetour(appart2019)

In [ ]:
predictor.predictionLinear(appart2019[appart2019.index==28], modele_lineaire_appart)

In [ ]:
maisons2019 = pd.read_csv(cr.CURATED_LOCAL_PATH + 'filteredmaisons2019.csv', sep=",")
appart2019 = pd.read_csv(cr.CURATED_LOCAL_PATH + 'filteredappart2019.csv', sep=",")


In [ ]:
splitter.split_datas('filteredmaisons2019.csv', 'Code postal', 'CPMaisons')

In [ ]:
splitter.split_datas('filteredappart2019.csv', 'Code postal', 'CPAppart')

In [ ]:
maisonsavendre = pd.read_csv(cr.CURATED_LOCAL_PATH+'CPMaisons/63.csv')

In [ ]:
predictor = ImmothepPrediction()

maisonsavendre = maisonsavendre.dropna()
modele, scaler = predictor.entrainementLR(maisonsavendre)

In [ ]:
predictor.predictionLR(maisonsavendre[maisonsavendre.index==123], modele, scaler)

In [ ]:
modeleL = predictor.trainLinearLeRetour(maisonsavendre)

In [ ]:
predictor.predictionLinear(maisonsavendre[maisonsavendre.index==123], modeleL)

In [ ]:
newdf = maisonsavendre.aggregate('Valeur fonciere', axis=1)